## 207. Course Schedule
- Description:
  <blockquote>
    There are a total of `numCourses` courses you have to take, labeled from `0` to `numCourses - 1`. You are given an array `prerequisites` where `prerequisites[i] = [a<sub>i</sub>, b<sub>i</sub>]` indicates that you **must** take course `b<sub>i</sub>` first if you want to take course `a<sub>i</sub>`.

    -   For example, the pair `[0, 1]`, indicates that to take course `0` you have to first take course `1`.

    Return `true` if you can finish all courses. Otherwise, return `false`.

    **Example 1:**

    ```
    Input: numCourses = 2, prerequisites = [[1,0]]
    Output: true
    Explanation: There are a total of 2 courses to take. 
    To take course 1 you should have finished course 0. So it is possible.

    ```

    **Example 2:**

    ```
    Input: numCourses = 2, prerequisites = [[1,0],[0,1]]
    Output: false
    Explanation: There are a total of 2 courses to take. 
    To take course 1 you should have finished course 0, and to take course 0 you should also have finished course 1. So it is impossible.

    ```

    **Constraints:**

    -   `1 <= numCourses <= 2000`
    -   `0 <= prerequisites.length <= 5000`
    -   `prerequisites[i].length == 2`
    -   `0 <= a<sub>i</sub>, b<sub>i</sub> < numCourses`
    -   All the pairs prerequisites\[i\] are **unique**.
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/course-schedule/description/)

- Topics: BFS+Topological Sort Using Kahn's Algorithm, DFS

- Difficulty: Medium / Hard

- Resources: example_resource_URL

### Solution 1, MOST OPTIMUM, BFS+Topological Sort Using Kahn's Algorithm

If we regard each course as a node and draw an edge from bi​ to ai​ for any prerequisite [ai​, bi​] (to indicate that course bi​ should be completed before taking course ai​), we get a directed graph.

If there is a cycle in this directed graph, it suggests that we will not be able to finish all of the courses. Otherwise, we can perform a topological sort of the graph to determine the order in which all of the courses can be finished.

As a result, the problem is reduced to determining whether a cycle occurs in a graph. If there is a cycle, we must return false. If not, we return true.

we can use Kahn's algorithm to get the topological ordering. Kahn’s algorithm works by keeping track of the number of incoming edges into each node (indegree). It works by repeatedly visiting the nodes with an indegree of zero and deleting all the edges associated with it leading to a decrement of indegree for the nodes whose incoming edges are deleted. This process continues until no elements with zero indegree can be found. The advantage of using Kahn's algorithm is that it also aids in the detection of graph cycles. If there is a cycle, the indegree of nodes in the cycle cannot be set to 0 due to cyclic dependency. We are unable to visit the cycle's nodes. So, if the number of visited nodes is less than the total number of nodes in the graph, we have a cycle.


---

Here, n be the number of courses and m be the size of prerequisites.

- Time Complexity: O(M+N)
  - Initializing the adj list takes O(m) time as we go through all the edges. The indegree array take O(n) time.
  - Each queue operation takes O(1) time, and a single node will be pushed once, leading to O(n) operations for n nodes. We iterate over the neighbors of each node that is popped out of the queue iterating over all the edges once. Since there are total of m edges, it would take O(m) time to iterate over the edges.
- Space Complexity: O(M+N)
  - The adj arrays takes O(m) space. The indegree array takes O(n) space.
  - The queue can have no more than n elements in the worst-case scenario. It would take up O(n) space in that case.

In [ ]:
class Solution:
    def canFinish(self, numCourses: int, prerequisites: List[List[int]]) -> bool:
        indegree = [0] * numCourses
        adj = [[] for _ in range(numCourses)]

        for prerequisite in prerequisites:
            adj[prerequisite[1]].append(prerequisite[0])
            indegree[prerequisite[0]] += 1

        queue = deque()
        
        for i in range(numCourses):
            if indegree[i] == 0:
                queue.append(i)

        nodesVisited = 0
        
        while queue:
            node = queue.popleft()
            nodesVisited += 1

            for neighbor in adj[node]:
                indegree[neighbor] -= 1
                if indegree[neighbor] == 0:
                    queue.append(neighbor)

        return nodesVisited == numCourses

### Solution 2, Depth First Search with visit & inStack lists to detect cycle

We can also use a depth-first search (DFS) traversal to detect a cycle in a directed graph.

In DFS, we use a recursive function to explore nodes as far as possible along each branch. Upon reaching the end of a branch, we backtrack to the previous node and continue exploring the next branches.

Once we encounter an unvisited node, we will take one of its neighbor nodes (if exists) as the next node on this branch. Recursively call the function to take the next node as the 'starting node' and solve the subproblem.

A node remains in the DFS recursion stack until all of its branches (all nodes in its subtree) have been explored. When we have examined all of a node's branches, i.e. visited all of the nodes in its subtree, the node is removed from the DFS recursive stack.

If the graph has a cycle, we must have a back edge connecting a node to one of its ancestors while traversing nodes in the DFS manner.

Otherwise, if a neighboring node is visited, it may or may not be an ancestor. If the neighboring node is an ancestor, i.e. there is a back edge, it means that we visited this ancestor node first in the DFS traversal, then visited and explored some other nodes, and eventually visited a node that connects back to the ancestor node. As we are still exploring the ancestor node's subtree while iterating over this path, hence this node must be in the current DFS recursive stack.

However, if a neighboring node is visited but not in the recursion stack, it signifies we have previously explored that node in a different branch, and it does not form a cycle in the current branch.

As a result, to detect the cycle we must keep track of the visited nodes (like in a normal DFS) and also the nodes in the function's recursion call stack for DFS traversal. The nodes in the stack store the current path that we are on. There is a cycle in the graph if a node is reached that is already in the recursion stack. We use a boolean array of length n to track which nodes are in the call stack so we can check if a node exists in O(1).

---

Here, n be the number of courses and m be the size of prerequisites.

- Time Complexity: O(M+N)
  - Initializing adj takes O(m) time as we go through all the edges.
  - Initializing the visit and inStack arrays take O(n) time each.
  - The dfs function handles each node once, which takes O(n) time in total. From each node, we iterate over all the outgoing edges, which further takes O(m) time to iterate over all the edges as there are a total of m edges.

- Space Complexity: O(M+N)
  - The adj arrays takes O(m) space.
  - The visit and inStack arrays take O(n) space each.
  - The recursion call stack used by dfs can have no more than n elements in the worst-case scenario. It would take up O(n) space in that case.

In [ ]:
class Solution:
    def dfs(self, node, adj, visit, inStack):
        # If the node is already in the stack, we have a cycle.
        if inStack[node]:
            return True
        
        if visit[node]:
            return False
        
        # Mark the current node as visited and part of current recursion stack.
        visit[node] = True
        inStack[node] = True
        
        for neighbor in adj[node]:
            if self.dfs(neighbor, adj, visit, inStack):
                return True
            
        # Remove the node from the stack.
        inStack[node] = False
        return False

    def canFinish(self, numCourses: int, prerequisites: List[List[int]]) -> bool:
        adj = [[] for _ in range(numCourses)]
        
        for prerequisite in prerequisites:
            adj[prerequisite[1]].append(prerequisite[0])

        visit = [False] * numCourses
        inStack = [False] * numCourses
        
        for i in range(numCourses):
            if self.dfs(i, adj, visit, inStack):
                return False
            
        return True

In [ ]:
# ALT DFS sol: DFS with an array storing 3 different states of a vertex

def buildAdjacencyList(self, n, edgesList):
    adjList = [[] for _ in range(n)]
    # c2 (course 2) is a prerequisite of c1 (course 1)
    # i.e c2c1 is a directed edge in the graph
    for c1, c2 in edgesList:
        adjList[c2].append(c1)
    return adjList

def canFinish(self, numCourses: int, prerequisites: List[List[int]]) -> bool:
    # build Adjacency list from Edges list
    adjList = self.buildAdjacencyList(numCourses, prerequisites)

    # Each vertex can have 3 different states:
    # state 0   : vertex is not visited. It's a default state.
    # state -1  : vertex is being processed. Either all of its descendants
    #             are not processed or it's still in the function call stack.
    # state 1   : vertex and all its descendants are processed.
    state = [0] * numCourses

    def hasCycle(v):
        if state[v] == 1:
            # This vertex is processed so we pass.
            return False
        if state[v] == -1:
            # This vertex is being processed and it means we have a cycle.
            return True

        # Set state to -1
        state[v] = -1

        for i in adjList[v]:
            if hasCycle(i):
                return True

        state[v] = 1
        return False

    # we traverse each vertex using DFS, if we find a cycle, stop and return
    for v in range(numCourses):
        if hasCycle(v):
            return False

    return True